In [1]:
import arcpy as ap
import os
import cmath
from collections import defaultdict

RuntimeError: NotInitialized

In [ ]:
# function to return coordinates from arc point object
def coordinates_of(point_object):
    x = round(point_object.X,5)
    y = round(point_object.Y,5)

    return x,y

In [ ]:
def x_meters_as_percentage(of_a_line, x_meters):
    of_line_length = of_a_line.length
    percentage_of_line = float(x_meters)/float(of_line_length)
    
    return percentage_of_line


def point_x_meters_from_the_start(of_a_line, x_meters):
    point_x_meters_from_start = of_a_line.positionAlongLine(x_meters_as_percentage(of_a_line, x_meters), True).firstPoint
    
    return coordinates_of(point_x_meters_from_start)


def point_x_meters_from_the_end(of_a_line, x_meters):
    point_10m_from_end = of_a_line.positionAlongLine(1 - x_meters_as_percentage(of_a_line, x_meters), True).firstPoint
    
    return coordinates_of(point_10m_from_end)

In [ ]:
# function to calculate the angle between two sets of coordinatesdef angle_between(first_point_coordinates, last_point_coordinates):
def angle_between(point_a_coordinates, point_b_coordinates): 
    x1, y1 = float(point_a_coordinates[0]), float(point_a_coordinates[1])
    x2, y2 = float(point_b_coordinates[0]), float(point_b_coordinates[1])

    polarcoords = cmath.polar(complex(x2 - x1, y2 - y1))
    
    if math.degrees(polarcoords[1]) <= 90:
        degrees = 90 - math.degrees(polarcoords[1])
    else:
        degrees = 450 - math.degrees(polarcoords[1])

    return round(degrees, 1)

In [ ]:
def GetDifference(b1, b2):
    r = (b2 - b1) % 360.0
    # Python modulus has same sign as divisor, which is positive here,
    # so no need to consider negative case
    if r >= 180.0:
        r -= 360.0
    return abs(r)

In [ ]:
def list_to_string(list_of):
    list_as_string = str(list_of).replace('[', '').replace(']','').replace(' ','')
    
    return list_as_string

In [ ]:
local_shapefiles_directory = r'F:\Conflating_Networks\network_shapefiles'
local_geodatabase = r'F:\Conflating_Networks\working_files.gdb'
local_dir = r'F:\Conflating_Networks'

#local_shapefiles_directory = os.getcwd() + '\\network_shapefiles'
#local_geodatabase = os.getcwd() + '\\working_files.gdb'

ap.env.workspace = local_shapefiles_directory
network_shapefiles = ap.ListFeatureClasses()

ap.SpatialReference(102382)
spatial_reference = ap.SpatialReference(102382)

ap.env.workspace = local_geodatabase

In [ ]:
# creating necesarry files in local geodatabase
if ap.Exists(local_geodatabase) == False:
    ap.CreateFileGDB_management(local_dir, 'working_files.gdb')
    
for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '//' + each_network).baseName
    ap.env.workspace = local_geodatabase
    
        
    if ap.Exists('%s_dissolved_to_line' % network_name) == False:       
        ap.Dissolve_management(local_shapefiles_directory + '//%s.shp' % network_name, 
                                   'in_memory\\%s_dissolved'% network_name,
                                   '',
                                   '',
                                   'MULTI_PART',
                                   'DISSOLVE_LINES')
                
        ap.FeatureToLine_management('in_memory\\%s_dissolved' % network_name,
                                            '%s_dissolved_to_line' % network_name)
        
        ap.Delete_management('in_memory\\%s_dissolved' % network_name)
    
    elif ap.Exists('%s_dissolved_to_line' % network_name) == True:
        for field in ['first_10m_angle', 'last_10m_angle', 'ORIG_FID']:
        #for field in ['start_to_end_angle','first_10m_angle', 'last_10m_angle', 'ORIG_FID']:
            
            try:
                ap.DeleteField_management('%s_dissolved_to_line_all_pts' % network_name, field)
            
            except:
                pass

In [ ]:
for each_network in network_shapefiles: 
    network_name = ap.Describe(local_shapefiles_directory + '//' + each_network).baseName
    
    for field in ['first_10m_angle', 'last_10m_angle']:
    #for field in ['start_to_end_angle','first_10m_angle', 'last_10m_angle', 'mid_pt']:

        if field not in [a.name for a in ap.ListFields('%s_dissolved_to_line' % network_name)]:

            ap.AddField_management('%s_dissolved_to_line' % network_name,
                                    '%s' % field,
                                   'STRING')
    
    with ap.da.UpdateCursor('%s_dissolved_to_line' % network_name,
                           ['SHAPE@',
                            'first_10m_angle',
                            'last_10m_angle',]) as angle_cursor:
        
    #with ap.da.UpdateCursor('%s_dissolved_to_line' % network_name,
     #                      ['SHAPE@',
    #                      'start_to_end_angle',
       #                     'first_10m_angle',
        #                    'last_10m_angle',
         #                   'mid_pt']) as angle_cursor:
        
        for row in angle_cursor:
            line_geometry = of_a_line = row[0]
            
            first_point = coordinates_of(line_geometry.firstPoint)
            last_point = coordinates_of(line_geometry.lastPoint)
            
            #line_angle = angle_between(first_point, last_point)
            
            point_10m_from_start = point_x_meters_from_the_start(of_a_line, 10)
            first_10m_angle = angle_between(first_point, point_10m_from_start)
            
            point_10m_from_end = point_x_meters_from_the_end(of_a_line,10)
            last_10m_angle = angle_between(last_point, point_10m_from_end)
            
            #mid_pt = str(coordinates_of(line_geometry.positionAlongLine(.5, True).firstPoint))
            
            row[1] = first_10m_angle
            row[2] = last_10m_angle
            #row[1] = line_angle
            #row[2] = first_10m_angle
            #row[3] = last_10m_angle
            #row[4] = mid_pt
            
            angle_cursor.updateRow(row)
    
    del angle_cursor    

In [ ]:
ap.SpatialJoin_analysis(target_features="%s_dissolved_to_line_all_pts" % network_name, 
                        join_features="%s_dissolved_to_line_start" % network_name, 
                        out_feature_class="%s_pts_to_start_pts" % network_name, 
                        join_operation="JOIN_ONE_TO_ONE", 
                        join_type="KEEP_ALL", 
                        field_mapping="""start_to_end_angle "start_to_end_angle" true true false 255 Text 0 0 ,Join,",",%s_dissolved_to_line_start,start_to_end_angle,-1,-1;first_10m_angle "first_10m_angle" true true false 255 Text 0 0 ,Join,",",%s_dissolved_to_line_start,first_10m_angle,-1,-1""" % (network_name, network_name), 
                        match_option="INTERSECT", 
                        search_radius="1 Centimeters", 
                        distance_field_name="")


ap.Delete_management("%s_pts_to_start_pts" % network_name)

for start_and_end in ['START', 'END']:
    ap.Delete_management('%s_dissolved_to_line_%s' % (network_name, start_and_end.lower()))

In [ ]:
if ap.Exists('%s_to_line' % network_name) == False:       
        ap.FeatureToLine_management(local_shapefiles_directory + '//%s.shp' % network_name,
                                    '%s_to_line' % network_name)

overpass_dictionary = {}

for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName
    
    with ap.da.SearchCursor('%s_to_line' % network_name,
                        'FID_%s' % network_name,
                        sql_clause = (None,'ORDER BY FID_%s' % network_name))as overpass_cursor:
        
        overpass_dictionary[network_name] = []
        last_feature_id = None
    
        for each_feature in overpass_cursor:
            feature_id = each_feature[0]
        
            if feature_id == last_feature_id:
                overpass_dictionary[network_name].append(last_feature_id)
                
            else:
                pass
            
            last_feature_id = feature_id
    try:
        ap.MakeFeatureLayer_management('%s_to_line' % network_name,
                                   '%s_to_line_lyr' % network_name)
    except:
        pass

In [ ]:
for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '\\' + each_network).baseName

    select_overpasses = '"FID_%s" IN %s' % (network_name, tuple(overpass_dictionary[network_name]))

    ap.SelectLayerByAttribute_management('%s_to_line_lyr' % network_name, 
                                             'NEW_SELECTION',
                                             select_overpasses)
    for start_and_end in ['START', 'END']:
        ap.FeatureVerticesToPoints_management ('%s_to_line' % network_name,
                                            '%s_to_line_%s' % (network_name, start_and_end.lower()), 
                                            start_and_end)

In [ ]:
ap.SpatialJoin_analysis(target_features="DTD_2010_Links_directions_dissolved_to_line_all_pts", 
                        join_features="DTD_2010_Links_directions_dissolved_to_line_start", 
                        out_feature_class="DTD_spatial_join_with_start1", 
                        join_operation="JOIN_ONE_TO_ONE", 
                        join_type="KEEP_ALL", 
                        field_mapping="""FID_DTD_2010_Links_directions_dissolved "FID_DTD_2010_Links_directions_dissolved" true true false 4 Long 0 0 ,First,#,DTD_2010_Links_directions_dissolved_to_line_all_pts,FID_DTD_2010_Links_directions_dissolved,-1,-1;start_to_end_angle "start_to_end_angle" true true false 255 Text 0 0 ,Join,",",DTD_2010_Links_directions_dissolved_to_line_start,start_to_end_angle,-1,-1;first_10m_angle "first_10m_angle" true true false 255 Text 0 0 ,Join,",",DTD_2010_Links_directions_dissolved_to_line_start,first_10m_angle,-1,-1;last_10m_angle "last_10m_angle" true true false 255 Text 0 0 ,Join,",",DTD_2010_Links_directions_dissolved_to_line_start,last_10m_angle,-1,-1""", 
                        match_option="INTERSECT", 
                        search_radius="1 Centimeters", 
                        distance_field_name="")





In [ ]:
b = [a.name for a in ap.ListFields('%s_dissolved_to_line_all_pts' % network_name)]

for x in b:
    print(x)